# **Brain Tumor Classification Using CNN**

### ***Author: Bilal Açıkgöz***

#### ***Import Library and Load Dataset***

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader, Dataset
from torchvision import datasets, models, transforms
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#### ***Visualization***

In [ ]:
dataset_path = "/kaggle/input/brain-mri-images-for-brain-tumor-detection"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
class_names = dataset.classes
class_names = class_names[1:]

In [ ]:
fig = plt.figure(figsize=(12, 9))
rows, cols = 4, 4

for i in range(1, rows * cols + 1):
    random_idx = torch.randint(0, len(dataset), size=[1]).item()
    img, label = dataset[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.permute(1, 2, 0))
    plt.suptitle("Visualization of Brain MRI", fontsize=20)
    if label < len(class_names):
        plt.title(class_names[label], fontsize=10)
    plt.axis(False);

#### ***Data Preprocessing and Prepare***

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label
    
def load_dataset_from_directory(directory):
    image_paths = []
    labels = []

    class_names = ['no', 'yes']
    class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}

    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            if img_path.endswith(('.png', '.jpg', 'JPG', '.jpeg')):  # Let's filter image file extensions
                image_paths.append(img_path)
                labels.append(class_to_idx[class_name])

    return image_paths, labels

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

image_paths, labels = load_dataset_from_directory(dataset_path)
dataset = CustomDataset(image_paths, labels, transform)

# Train-validation and dataset split
train_size = int(0.7 * len(dataset))  # %80 Train
val_test_size = len(dataset) - train_size  # %20 Validation-Test
train_dataset, val_test_dataset = random_split(dataset, [train_size, val_test_size])

val_size = int(0.6 * len(val_test_dataset)) # Val
test_size = len(val_test_dataset) - val_size # Test
val_dataset, test_dataset = random_split(val_test_dataset, [val_size, test_size])

# Load dataset with Dataloader with batch size
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
for i in torch.arange(253):
    print(image_paths[i], "\t", labels[i])

In [ ]:
# Check dataset dimensions
print(f"Total data: {len(dataset)}")
print(f"Train data dimension: {len(train_dataset)}")
print(f"Val data dimension: {len(val_dataset)}")
print(f"Test data dimension: {len(test_dataset)}")

#### ***Build the ResNet Model***

In [ ]:
try:
    # If multiple GPUs are available, use them
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_gpus = torch.cuda.device_count()
    print(f"Using {num_gpus} GPUs for training." if num_gpus > 1 else f"Using {torch.cuda.get_device_name()} for training." if torch.cuda.is_available() else "Using CPU for training.")
except:
    print("No GPU found. Using CPU.")
    device = torch.device("cpu")

In [ ]:
num_classes = len(class_names)
resnet152_model = models.resnet152(pretrained=True)
resnet152_model.fc = nn.Linear(resnet152_model.fc.in_features, num_classes).to(device)

In [ ]:
resnet152_model

In [ ]:
if num_gpus > 1:
    resnet152_model = nn.DataParallel(resnet152_model).to(device)

#### ***Model Training***

In [ ]:
# Training function
def train(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    for batch_inputs, batch_labels in train_loader:
        batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
    average_loss = total_loss / len(train_loader)
    accuracy = correct / total * 100
    return average_loss, accuracy

# Validation function
def validate(model, val_loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_inputs, batch_labels in val_loader:
            batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()
    average_loss = total_loss / len(val_loader)
    accuracy = correct / total * 100
    return accuracy, average_loss

In [ ]:
# Define criterion, optimizer and early_stopping
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet152_model.parameters(), lr=1e-4, weight_decay=1e-3)

# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(resnet152_model, train_loader, criterion, optimizer)
    val_accuracy, val_loss = validate(resnet152_model, val_loader, criterion)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

print('Training was completed')

In [ ]:
torch.save(resnet152_model, "resnet_model.pth")

#### ***Model Evaluating and Visualization***

In [ ]:
# Test Func
def test(model, test_loader):
    model.eval()  # Evaluation mode
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # Disable gradient calculation
        for batch_inputs, batch_labels in test_loader:
            batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)
            outputs = model(batch_inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())
    
    return all_preds, all_labels

# Testing Model
all_preds, all_labels = test(resnet152_model, test_loader)

# Calculate test accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Precision, Recall, F1-Score 
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")

In [ ]:
# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Class")
plt.ylabel("Real Class")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Visualization of Misclassifications:
def plot_misclassified(model, test_loader, class_names):
    model.eval()
    misclassified = []
    
    with torch.no_grad():
        for batch_inputs, batch_labels in test_loader:
            batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)
            outputs = model(batch_inputs)
            _, predicted = torch.max(outputs, 1)
            
            # Catch of wrong classifications
            for i in range(len(predicted)):
                if predicted[i] != batch_labels[i]:
                    misclassified.append((batch_inputs[i].cpu(), predicted[i].cpu(), batch_labels[i].cpu()))
    
    fig = plt.figure(figsize=(12, 5))
    rows, cols = 1, 3  # Let's show the first 9 wrong examples
    for i in range(1, rows * cols + 1):
        img, predicted, actual = misclassified[i - 1]
        fig.add_subplot(rows, cols, i)
        plt.imshow(img.permute(1, 2, 0))
        plt.title(f"Real: {class_names[actual]} \nPredict: {class_names[predicted]}")
        plt.axis(False)

    plt.suptitle("Misclassified Samples", fontsize=16)
    plt.show()

plot_misclassified(resnet152_model, test_loader, class_names)

#### ***Prediction***

In [ ]:
def predict_and_visualize_from_test(test_loader, model, class_names, num_images=5):
    model.eval()
    images_so_far = 0
    fig = plt.figure(figsize=(15, 15))

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # Visualize images within each batch
            for i in range(inputs.size()[0]):
                if images_so_far == num_images:
                    return
                images_so_far += 1
                
                img = inputs[i].cpu().permute(1, 2, 0)  # Move image to CPU and edit channels
                img = img * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # Reverse normalization
                
                fig.add_subplot(num_images // 2, 2, images_so_far)
                plt.imshow(img)
                plt.title(f"Predict: {class_names[preds[i]]} \nReal: {class_names[labels[i]]}")
                plt.axis('off')

    plt.show()

predict_and_visualize_from_test(test_loader, resnet152_model, class_names, num_images=10)

#### ***Conclusion***

In this notebook, we have successfully built a brain tumor binary classification model using deep learning techniques. By applying transfer learning to the Resnet152 model and through appropriate model evaluation, we were able to detect whether there is a brain tumor or not with the model we trained. We can use this model in different applications to detect brain tumors.